In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
df.head()

,review,label
0,I think they really let the quality of the DVD...,0
1,I'm sorry but this is just awful. I have told ...,0
2,"The Japenese sense of pacing, editing and musi...",0
3,"In the '60's/'70's, David Jason was renowned f...",1
4,"""Hail The Woman"" is one of the most moving fil...",1


In [2]:
from sklearn.metrics import f1_score

In [3]:
labels = np.array(df["label"])

Создание доп таблицы: ? ! кол-во слов

In [4]:
ask = np.zeros(len(df))
for i in range (len(ask)):
    ask[i] = df["review"][i].count('?')
ask1 = np.zeros(len(test))
for i in range (len(ask1)):
    ask1[i] = test["review"][i].count('?')

In [5]:
add = pd.DataFrame(ask,columns = ['?'])

In [6]:
add_t = pd.DataFrame(ask1,columns = ['?'])

In [7]:
v = np.zeros(len(df))
for i in range (len(v)):
    v[i] = df["review"][i].count('!')
v1 = np.zeros(len(test))
for i in range (len(v1)):
    v1[i] = test["review"][i].count('!')

In [8]:
add['!'] = pd.Series(v)
add_t['!'] = pd.Series(v1)

In [9]:
a2 = np.zeros(len(df))
for i in range (len(a2)):
    a2[i] = len(df["review"][i])
at2 = np.zeros(len(test))
for i in range (len(at2)):
    at2[i] = len(test["review"][i])

In [10]:
add['lens'] = pd.Series(a2)
add_t['lens'] = pd.Series(at2)

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

add = scaler.fit_transform(add)
add_t = scaler.fit_transform(add_t)
#add

In [12]:
add = pd.DataFrame(add,columns=('?','!','lens'))

In [13]:
add_t = pd.DataFrame(add_t,columns=('?','!','lens'))
#add_t

In [14]:
rates = np.zeros(len(df))
for i in range(len(rates)):
    if (df["review"][i].count('6/10') + df["review"][i].count('7/10') + df["review"][i].count('8/10') + df["review"][i].count('9/10')+df["review"][i].count('10/10') > 0):
        rates[i] = 1
    else:
        if (df["review"][i].count('0/10') + df["review"][i].count('1/10') + df["review"][i].count('2/10') + df["review"][i].count('3/10')+df["review"][i].count('4/10') > 0):
            rates[i] = 0
        else:
            rates[i] = 1/2

In [15]:
len(np.where(rates < 1/2)[0])

956

In [16]:
rates_t = np.zeros(len(test))
for i in range(len(rates_t)):
    if (test["review"][i].count('6/10') + test["review"][i].count('7/10') + test["review"][i].count('8/10') + test["review"][i].count('9/10')+test["review"][i].count('10/10') > 0):
        rates_t[i] = 1
    else:
        if (test["review"][i].count('0/10') + test["review"][i].count('1/10') + test["review"][i].count('2/10') + test["review"][i].count('3/10')+test["review"][i].count('4/10') > 0):
            rates_t[i] = 0
        else:
            rates_t[i] = 1/2

In [17]:
len(np.where(rates_t < 1/2)[0])

249

Перевод слов в вектора

In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

en_stop = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 

def tokenize(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if
              re.match(r'[^\W\d]*$', t) and (len(t) > 2) and (t not in en_stop)]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

tokens = df['review'].apply(tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/daniilzakharov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daniilzakharov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daniilzakharov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [230]:
#import gensim
from gensim.models import Word2Vec

model = Word2Vec(tokens, window=3, min_count=5)

Перевод текста в вектор средний по словам в нём

In [231]:
def encodemin(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.index_to_key])
    return np.min(x, axis=0)

ftsmin = np.array([encodemin(t) for t in tokens])
ftsmin.shape

(40000, 100)

In [232]:
def encodemax(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.index_to_key])
    return np.max(x, axis=0)

ftsmax = np.array([encodemax(t) for t in tokens])
ftsmax.shape

(40000, 100)

In [233]:
def encode(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.index_to_key])
    return np.mean(x, axis=0)

fts = np.array([encode(t) for t in tokens])
fts.shape

(40000, 100)

In [25]:
from sklearn.model_selection import train_test_split

In [234]:
mainmax = pd.DataFrame(ftsmax)

In [235]:
mainmin = pd.DataFrame(ftsmin)

In [236]:
main = pd.DataFrame(fts)
main

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.591807,0.412227,-0.542035,0.235885,0.104132,-0.266516,-0.151623,0.747705,-0.491476,-0.921788,...,0.750401,0.002443,-0.358763,-0.058084,0.330841,0.253669,-0.337376,-0.003289,-0.099390,0.081262
1,-0.561351,0.539296,-0.232209,0.017927,0.295141,-0.529878,-0.322349,0.504917,-0.184213,-0.902645,...,0.657582,0.318418,-0.342166,-0.195031,0.356839,-0.012878,-0.207628,-0.119354,-0.248098,-0.024733
2,-0.664570,0.464939,-0.340457,-0.091051,0.005002,-0.515267,-0.082001,0.493307,-0.265265,-0.685733,...,0.676701,0.200060,-0.352417,0.068066,0.565982,0.016959,-0.493918,-0.072227,-0.121789,0.068586
3,-0.152027,0.341520,-0.116394,0.070313,-0.078539,-0.322824,-0.012664,0.391128,-0.158454,-0.564159,...,0.146705,0.221364,-0.261125,-0.064428,0.239386,0.196713,-0.032280,-0.036533,0.172501,-0.068884
4,-0.238262,0.324238,-0.217468,0.280634,-0.047191,-0.554459,-0.017087,0.435945,-0.343209,-0.658837,...,0.355283,0.192587,-0.188468,0.152330,0.483326,0.164682,-0.201933,-0.162726,-0.003239,-0.002631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,-0.325537,0.395521,-0.091214,0.134563,-0.017822,-0.500446,-0.172351,0.707692,-0.150523,-0.528643,...,0.483865,0.146623,-0.187299,0.043661,0.248359,0.199509,-0.235859,-0.154411,-0.138025,-0.127265
39996,-0.414435,0.448172,-0.341371,0.036420,-0.018985,-0.482495,-0.136398,0.583454,-0.224770,-0.705551,...,0.831797,0.022653,-0.376826,0.013646,0.460591,0.107461,-0.075953,-0.074680,-0.047151,-0.090945
39997,-0.420794,0.516332,-0.284983,0.123140,-0.000968,-0.410312,0.183259,0.430604,-0.188347,-0.672189,...,0.495289,0.145446,-0.124174,-0.044993,0.236162,0.133464,-0.024950,-0.188739,-0.064351,-0.027500
39998,-0.433937,0.879612,-0.465086,0.016599,0.007571,-0.308500,0.029983,0.500054,-0.164624,-0.930576,...,0.756202,0.212626,-0.407688,-0.153975,0.225495,0.153089,-0.259771,-0.215287,0.010675,0.003528


Создаём для теста то же самое

In [29]:
test_tokens = test['review'].apply(tokenize)

In [237]:
test_ftsmin = np.array([encodemin(t) for t in test_tokens])
test_ftsmin.shape
test_ftsmin

array([[-2.580983 , -1.3969651, -1.8512778, ..., -1.6710719, -1.8479671,
        -2.0984654],
       [-2.1739564, -1.1485314, -1.9737214, ..., -2.4744458, -1.3834903,
        -1.7516061],
       [-2.908062 , -2.0748682, -3.4307623, ..., -1.7065064, -1.7632034,
        -2.0984654],
       ...,
       [-3.047236 , -1.3791972, -3.283116 , ..., -2.255286 , -2.275498 ,
        -2.1038988],
       [-2.1739564, -1.3969651, -1.848078 , ..., -1.7065064, -1.8479671,
        -2.042474 ],
       [-2.580983 , -1.053847 , -3.283116 , ..., -1.9321309, -1.9221258,
        -2.5095036]], dtype=float32)

In [238]:
test_ftsmax = np.array([encodemax(t) for t in test_tokens])
test_ftsmax.shape
test_ftsmax

array([[1.6494902, 3.7746737, 1.6929892, ..., 1.8690282, 2.169898 ,
        2.0484333],
       [1.120752 , 3.0250616, 1.3444445, ..., 2.018312 , 1.068491 ,
        2.2281256],
       [2.2134454, 3.0250616, 1.8803526, ..., 2.5058267, 2.3891582,
        2.2281256],
       ...,
       [1.4917171, 3.7746737, 1.7233075, ..., 1.7584566, 2.3064806,
        2.4554012],
       [1.0770214, 3.7746737, 1.1496803, ..., 1.8361859, 2.169898 ,
        1.7406156],
       [1.7940805, 2.5088475, 1.8716601, ..., 1.9193672, 2.167308 ,
        2.083212 ]], dtype=float32)

In [239]:
test_fts = np.array([encode(t) for t in test_tokens])
test_fts.shape
test_fts

array([[-0.42208824,  0.6239239 , -0.13489456, ...,  0.07511898,
         0.03168654, -0.14893948],
       [-0.44892016,  0.41161826, -0.35673746, ..., -0.19068535,
        -0.12135475,  0.15263282],
       [-0.39235067,  0.5208438 , -0.1909564 , ...,  0.02020677,
         0.04373298, -0.1524283 ],
       ...,
       [-0.30695882,  0.36858666, -0.23334189, ..., -0.12655981,
         0.07442541,  0.04063143],
       [-0.34267998,  0.26696348, -0.09615622, ..., -0.10199364,
        -0.03423222, -0.07072382],
       [-0.2240854 ,  0.29064342, -0.16506012, ..., -0.1458066 ,
         0.16348311, -0.10075652]], dtype=float32)

In [240]:
mainT=pd.DataFrame(test_fts)
mainT

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.422088,0.623924,-0.134895,-0.150155,-0.003067,-0.356807,0.017889,0.374306,-0.075294,-0.723016,...,0.378502,0.401696,-0.461866,-0.073895,0.455382,0.230552,0.095048,0.075119,0.031687,-0.148939
1,-0.448920,0.411618,-0.356737,0.110654,0.058743,-0.595456,-0.110939,0.521182,-0.152328,-0.720275,...,0.589642,0.156409,-0.371279,-0.022882,0.290206,0.237417,-0.427480,-0.190685,-0.121355,0.152633
2,-0.392351,0.520844,-0.190956,0.059784,-0.038225,-0.412573,0.072141,0.380138,-0.211477,-0.651279,...,0.438223,0.246256,-0.262773,0.072372,0.415889,0.236064,0.024850,0.020207,0.043733,-0.152428
3,-0.289658,0.418946,-0.278016,0.057252,0.037030,-0.454300,-0.038163,0.492555,-0.204482,-0.675188,...,0.405425,0.134241,-0.306946,0.052226,0.369333,0.141809,-0.125700,-0.091181,0.050232,-0.069573
4,-0.526020,0.570565,-0.370353,-0.012514,0.027801,-0.276075,0.084151,0.519893,-0.301597,-0.556995,...,0.764677,0.039534,-0.373652,0.008309,0.425068,0.037079,0.011471,-0.052075,-0.127436,-0.122433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.513173,0.669504,-0.193019,0.080172,-0.174542,-0.482258,-0.075219,0.388090,-0.237690,-0.713239,...,0.563218,0.256014,-0.422350,-0.033701,0.294768,0.258762,-0.556602,-0.141297,-0.048803,0.073185
9996,-0.278203,0.118832,0.046464,0.021633,-0.015761,-0.449065,0.216599,0.531371,-0.190271,-0.481586,...,0.205994,0.186770,-0.095967,0.249612,0.427946,0.280493,0.062318,-0.251321,0.094970,-0.245426
9997,-0.306959,0.368587,-0.233342,0.009264,-0.054013,-0.465737,-0.070932,0.556825,-0.231495,-0.611901,...,0.272401,0.241586,-0.415229,-0.001153,0.274999,0.065277,0.024031,-0.126560,0.074425,0.040631
9998,-0.342680,0.266963,-0.096156,-0.062876,-0.001338,-0.486167,-0.019108,0.476763,-0.234395,-0.396763,...,0.301378,0.037032,-0.170152,-0.023954,0.393987,0.133758,0.049314,-0.101994,-0.034232,-0.070724


In [241]:
mainTmax=pd.DataFrame(test_ftsmax)
mainTmax

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.649490,3.774674,1.692989,1.832493,2.472983,1.045970,1.592506,2.465509,1.372792,1.311762,...,2.334547,2.195520,0.928987,2.154305,2.393270,2.347820,1.654359,1.869028,2.169898,2.048433
1,1.120752,3.025062,1.344445,1.698329,1.355498,0.720966,1.447974,2.143462,1.439441,1.311762,...,2.569202,1.434698,1.431136,1.491210,2.721921,1.777075,0.692540,2.018312,1.068491,2.228126
2,2.213445,3.025062,1.880353,2.346940,1.842585,1.511473,2.979753,2.465509,1.736100,1.341959,...,2.885213,2.525924,1.794665,2.252112,2.721921,2.347820,2.434946,2.505827,2.389158,2.228126
3,2.585551,3.774674,1.871660,2.157986,2.903640,1.511473,1.716422,2.836618,1.520836,1.341959,...,3.072423,2.195520,2.182866,2.239624,2.676615,2.545439,2.434946,2.505827,2.524778,2.048433
4,1.491717,3.774674,1.359197,1.832493,1.745237,1.788455,2.124327,3.439366,1.736100,1.503689,...,3.072423,1.850804,0.805255,2.200300,3.052320,1.425656,1.963322,2.440057,1.332239,2.038542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.641036,3.025062,1.489226,1.698329,1.172903,1.412469,1.652665,1.827192,1.372792,1.111186,...,2.885213,2.525924,3.020535,1.842870,1.916826,2.239774,1.661684,1.869028,1.495963,1.788894
9996,1.440456,1.409754,1.145716,1.874662,1.574337,1.179283,1.979858,2.937500,1.442099,1.395098,...,2.135296,2.195520,2.134080,1.998402,2.628554,1.967128,2.434946,1.836186,2.389158,2.511229
9997,1.491717,3.774674,1.723307,2.404067,2.903640,1.859905,1.954281,3.009267,1.437022,0.883982,...,2.832394,2.127637,1.418715,2.239624,2.064272,2.347820,2.494396,1.758457,2.306481,2.455401
9998,1.077021,3.774674,1.149680,1.995631,1.574337,0.901966,2.979753,3.009267,1.437022,1.395098,...,2.623274,2.195520,1.293943,1.660877,2.064272,1.442362,1.576990,1.836186,2.169898,1.740616


In [242]:
mainTmin=pd.DataFrame(test_ftsmin)
mainTmin

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-2.580983,-1.396965,-1.851278,-2.845459,-1.638530,-2.481742,-1.553009,-1.566970,-1.930569,-3.136898,...,-1.298195,-0.724361,-3.025336,-2.038851,-1.277535,-1.952883,-1.666014,-1.671072,-1.847967,-2.098465
1,-2.173956,-1.148531,-1.973721,-1.629371,-1.172922,-2.588961,-1.553009,-1.566970,-1.444633,-2.678741,...,-0.440838,-1.060694,-2.928394,-1.921575,-1.277535,-1.952883,-3.265694,-2.474446,-1.383490,-1.751606
2,-2.908062,-2.074868,-3.430762,-1.753898,-1.734006,-2.192731,-2.215030,-1.544352,-1.899911,-3.136898,...,-1.611569,-1.304402,-2.541664,-2.038851,-2.152296,-2.143595,-2.481215,-1.706506,-1.763203,-2.098465
3,-3.047236,-1.553577,-3.430762,-2.845459,-1.691360,-2.622186,-2.897722,-1.566970,-3.256143,-3.136898,...,-1.509303,-2.359668,-3.211020,-2.420814,-2.067665,-1.952883,-2.585572,-1.706506,-1.861337,-2.509504
4,-2.580983,-1.155652,-2.885654,-2.845459,-1.691360,-1.922402,-2.215030,-0.803439,-3.142730,-3.136898,...,-1.436539,-1.585095,-3.211020,-2.067279,-0.720920,-1.952883,-2.238898,-1.310795,-1.847967,-1.898332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-3.047236,-0.975113,-2.360014,-2.137521,-1.710124,-2.588961,-1.599298,-1.350230,-2.295771,-2.464462,...,-1.285631,-1.627701,-3.211020,-2.802808,-1.214533,-1.952883,-3.265694,-1.390033,-1.567042,-1.286852
9996,-1.911717,-1.396965,-1.851278,-1.629371,-1.624243,-2.481742,-1.553009,-1.566970,-2.024703,-2.676080,...,-1.509303,-0.989552,-3.211020,-1.921575,-1.277535,-0.970807,-2.736080,-2.113840,-1.770322,-2.031630
9997,-3.047236,-1.379197,-3.283116,-2.845459,-1.744129,-3.018056,-2.215030,-1.758490,-3.142730,-2.808486,...,-1.191121,-2.167803,-3.211020,-1.866963,-1.982158,-1.949728,-2.005701,-2.255286,-2.275498,-2.103899
9998,-2.173956,-1.396965,-1.848078,-2.845459,-1.785803,-2.481742,-1.535157,-1.356289,-2.687672,-2.808486,...,-1.456240,-1.607801,-3.025336,-2.156057,-1.086578,-1.952883,-2.056295,-1.706506,-1.847967,-2.042474


Разбиваем исходный датасет на трейн и тест

Добавление максимумов и минимумов

In [36]:
main

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.527165,0.318891,-0.539703,0.177884,-0.000614,-0.311103,-0.133428,0.831365,-0.521884,-0.916444,...,0.833589,0.022819,-0.388610,-0.052558,0.290471,0.287679,-0.389113,0.058967,-0.167193,0.271011
1,-0.545251,0.455068,-0.153106,0.103078,0.035308,-0.560647,-0.260883,0.497217,-0.163050,-0.896847,...,0.769409,0.257872,-0.276904,-0.149941,0.405372,0.000868,-0.120377,-0.096503,-0.236354,0.010060
2,-0.582557,0.446860,-0.288889,-0.032594,-0.132014,-0.522205,-0.035533,0.457761,-0.301574,-0.580363,...,0.671014,0.213228,-0.463490,-0.016136,0.619948,0.053613,-0.511097,0.010737,-0.132902,0.211562
3,-0.137423,0.278830,-0.018032,0.094090,-0.073242,-0.331278,0.026087,0.387974,-0.162785,-0.571416,...,0.200555,0.200816,-0.261931,-0.044498,0.234964,0.183833,-0.011739,-0.019069,0.110034,0.000655
4,-0.226351,0.189697,-0.216148,0.275151,-0.146258,-0.505466,0.039163,0.452622,-0.379290,-0.608569,...,0.350111,0.178537,-0.227001,0.179847,0.460390,0.212797,-0.138843,-0.143178,0.019891,0.120665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,-0.266279,0.348980,-0.077698,0.156306,-0.093855,-0.511772,-0.144109,0.689447,-0.114481,-0.536067,...,0.524881,0.072196,-0.213227,0.054734,0.339641,0.228714,-0.141016,-0.125116,-0.132959,-0.030900
39996,-0.384611,0.333402,-0.222750,0.022105,-0.231801,-0.443520,-0.103228,0.591732,-0.293793,-0.633671,...,0.868959,0.072136,-0.344132,-0.002489,0.500150,0.188006,-0.155242,-0.027173,0.004048,-0.011392
39997,-0.382089,0.474738,-0.217713,0.137511,-0.129702,-0.472200,0.160662,0.430111,-0.213829,-0.666216,...,0.514529,0.118649,-0.185415,0.009073,0.266012,0.158236,-0.004382,-0.207819,-0.091471,0.069712
39998,-0.367494,0.753373,-0.352407,0.036115,-0.182254,-0.353057,0.154910,0.518902,-0.110480,-0.820676,...,0.782343,0.214406,-0.495815,-0.239210,0.456869,0.285964,-0.220248,-0.156893,-0.105643,0.213866


In [249]:
X_train, X_test, y_train, y_test = train_test_split(main, df.label.values,
                                                    test_size=0.2, shuffle=True)

Делаем первую регрессию

In [250]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)

In [251]:
from sklearn.metrics import classification_report

predicts = clf.predict(X_train)
print('Train\n', classification_report(y_train, predicts))

predicts = clf.predict(X_test)
print('Test\n', classification_report(y_test, predicts))

Train
               precision    recall  f1-score   support

           0       0.86      0.85      0.85     16065
           1       0.85      0.86      0.85     15935

    accuracy                           0.85     32000
   macro avg       0.85      0.85      0.85     32000
weighted avg       0.85      0.85      0.85     32000

Test
               precision    recall  f1-score   support

           0       0.86      0.84      0.85      4002
           1       0.85      0.86      0.85      3998

    accuracy                           0.85      8000
   macro avg       0.85      0.85      0.85      8000
weighted avg       0.85      0.85      0.85      8000



In [252]:
predicts = clf.predict(mainT)

К-минс, можно Грид для него

In [111]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [421]:
#parameters = {'n_clusters':(np.arange(100,102)) }
#KMeans(n_clusters=5000, max_iter=10, n_jobs=8)
#grid = GridSearchCV(KMeans(),parameters,cv=5)

#grid.fit(X_train,y_train)

#Alpha, best_score = grid.best_params_, grid.best_score_

In [422]:
#Alpha, best_score

In [253]:
kmeans =  MiniBatchKMeans(n_clusters = 200)
kmeans.fit(main)
kmeans.inertia_

43003.01926422119

In [254]:
kmpos = np.zeros(len(df)) #сюда запишем итоговые значения параметра
for j in range(200): #считаем что кластеров 200
    pos = 0 #счетчик единиц в кластере
    for i in np.where(kmeans.labels_ == j)[0]:
        if labels[i] == 1:
            pos = pos+1
    for i in np.where(kmeans.labels_ == j)[0]:
        kmpos[i] = pos/len(np.where(kmeans.labels_ == j)[0])

In [255]:
kmpos

array([0.40343348, 0.06910569, 0.32833333, ..., 0.54740608, 0.37280702,
       0.4922049 ])

In [256]:
main['kmpos'] = kmpos
main

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,kmpos
0,-0.591807,0.412227,-0.542035,0.235885,0.104132,-0.266516,-0.151623,0.747705,-0.491476,-0.921788,...,0.002443,-0.358763,-0.058084,0.330841,0.253669,-0.337376,-0.003289,-0.099390,0.081262,0.403433
1,-0.561351,0.539296,-0.232209,0.017927,0.295141,-0.529878,-0.322349,0.504917,-0.184213,-0.902645,...,0.318418,-0.342166,-0.195031,0.356839,-0.012878,-0.207628,-0.119354,-0.248098,-0.024733,0.069106
2,-0.664570,0.464939,-0.340457,-0.091051,0.005002,-0.515267,-0.082001,0.493307,-0.265265,-0.685733,...,0.200060,-0.352417,0.068066,0.565982,0.016959,-0.493918,-0.072227,-0.121789,0.068586,0.328333
3,-0.152027,0.341520,-0.116394,0.070313,-0.078539,-0.322824,-0.012664,0.391128,-0.158454,-0.564159,...,0.221364,-0.261125,-0.064428,0.239386,0.196713,-0.032280,-0.036533,0.172501,-0.068884,0.747748
4,-0.238262,0.324238,-0.217468,0.280634,-0.047191,-0.554459,-0.017087,0.435945,-0.343209,-0.658837,...,0.192587,-0.188468,0.152330,0.483326,0.164682,-0.201933,-0.162726,-0.003239,-0.002631,0.841808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,-0.325537,0.395521,-0.091214,0.134563,-0.017822,-0.500446,-0.172351,0.707692,-0.150523,-0.528643,...,0.146623,-0.187299,0.043661,0.248359,0.199509,-0.235859,-0.154411,-0.138025,-0.127265,0.730337
39996,-0.414435,0.448172,-0.341371,0.036420,-0.018985,-0.482495,-0.136398,0.583454,-0.224770,-0.705551,...,0.022653,-0.376826,0.013646,0.460591,0.107461,-0.075953,-0.074680,-0.047151,-0.090945,0.234783
39997,-0.420794,0.516332,-0.284983,0.123140,-0.000968,-0.410312,0.183259,0.430604,-0.188347,-0.672189,...,0.145446,-0.124174,-0.044993,0.236162,0.133464,-0.024950,-0.188739,-0.064351,-0.027500,0.547406
39998,-0.433937,0.879612,-0.465086,0.016599,0.007571,-0.308500,0.029983,0.500054,-0.164624,-0.930576,...,0.212626,-0.407688,-0.153975,0.225495,0.153089,-0.259771,-0.215287,0.010675,0.003528,0.372807


In [257]:
main = main.join(add)

теперь для теста

In [258]:
kmeans.fit(mainT)
kmeans.inertia_

10902.315864562988

In [259]:
kmpos_t = np.zeros(len(test)) #сюда запишем итоговые значения параметра
for j in range(200): #считаем что кластеров 200
    pos = 0 #счетчик единиц в кластере
    for i in np.where(kmeans.labels_ == j)[0]:
        if predicts[i] == 1:
            pos = pos+1
    for i in np.where(kmeans.labels_ == j)[0]:
        kmpos_t[i] = pos/len(np.where(kmeans.labels_ == j)[0])

In [260]:
kmpos_t

array([0.15668203, 0.81481481, 0.15668203, ..., 0.60516605, 0.88157895,
       0.69927536])

In [261]:
mainT['kmpos'] = kmpos_t
mainT

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,kmpos
0,-0.422088,0.623924,-0.134895,-0.150155,-0.003067,-0.356807,0.017889,0.374306,-0.075294,-0.723016,...,0.401696,-0.461866,-0.073895,0.455382,0.230552,0.095048,0.075119,0.031687,-0.148939,0.156682
1,-0.448920,0.411618,-0.356737,0.110654,0.058743,-0.595456,-0.110939,0.521182,-0.152328,-0.720275,...,0.156409,-0.371279,-0.022882,0.290206,0.237417,-0.427480,-0.190685,-0.121355,0.152633,0.814815
2,-0.392351,0.520844,-0.190956,0.059784,-0.038225,-0.412573,0.072141,0.380138,-0.211477,-0.651279,...,0.246256,-0.262773,0.072372,0.415889,0.236064,0.024850,0.020207,0.043733,-0.152428,0.156682
3,-0.289658,0.418946,-0.278016,0.057252,0.037030,-0.454300,-0.038163,0.492555,-0.204482,-0.675188,...,0.134241,-0.306946,0.052226,0.369333,0.141809,-0.125700,-0.091181,0.050232,-0.069573,0.405204
4,-0.526020,0.570565,-0.370353,-0.012514,0.027801,-0.276075,0.084151,0.519893,-0.301597,-0.556995,...,0.039534,-0.373652,0.008309,0.425068,0.037079,0.011471,-0.052075,-0.127436,-0.122433,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.513173,0.669504,-0.193019,0.080172,-0.174542,-0.482258,-0.075219,0.388090,-0.237690,-0.713239,...,0.256014,-0.422350,-0.033701,0.294768,0.258762,-0.556602,-0.141297,-0.048803,0.073185,0.590476
9996,-0.278203,0.118832,0.046464,0.021633,-0.015761,-0.449065,0.216599,0.531371,-0.190271,-0.481586,...,0.186770,-0.095967,0.249612,0.427946,0.280493,0.062318,-0.251321,0.094970,-0.245426,0.681159
9997,-0.306959,0.368587,-0.233342,0.009264,-0.054013,-0.465737,-0.070932,0.556825,-0.231495,-0.611901,...,0.241586,-0.415229,-0.001153,0.274999,0.065277,0.024031,-0.126560,0.074425,0.040631,0.605166
9998,-0.342680,0.266963,-0.096156,-0.062876,-0.001338,-0.486167,-0.019108,0.476763,-0.234395,-0.396763,...,0.037032,-0.170152,-0.023954,0.393987,0.133758,0.049314,-0.101994,-0.034232,-0.070724,0.881579


In [262]:
mainT = mainT.join(add_t)
mainT

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,kmpos,?,!,lens
0,-0.422088,0.623924,-0.134895,-0.150155,-0.003067,-0.356807,0.017889,0.374306,-0.075294,-0.723016,...,0.455382,0.230552,0.095048,0.075119,0.031687,-0.148939,0.156682,0.000000,0.00625,0.100303
1,-0.448920,0.411618,-0.356737,0.110654,0.058743,-0.595456,-0.110939,0.521182,-0.152328,-0.720275,...,0.290206,0.237417,-0.427480,-0.190685,-0.121355,0.152633,0.814815,0.000000,0.00625,0.053334
2,-0.392351,0.520844,-0.190956,0.059784,-0.038225,-0.412573,0.072141,0.380138,-0.211477,-0.651279,...,0.415889,0.236064,0.024850,0.020207,0.043733,-0.152428,0.156682,0.026667,0.00000,0.386849
3,-0.289658,0.418946,-0.278016,0.057252,0.037030,-0.454300,-0.038163,0.492555,-0.204482,-0.675188,...,0.369333,0.141809,-0.125700,-0.091181,0.050232,-0.069573,0.405204,0.013333,0.00000,0.366509
4,-0.526020,0.570565,-0.370353,-0.012514,0.027801,-0.276075,0.084151,0.519893,-0.301597,-0.556995,...,0.425068,0.037079,0.011471,-0.052075,-0.127436,-0.122433,0.375000,0.000000,0.03750,0.094636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.513173,0.669504,-0.193019,0.080172,-0.174542,-0.482258,-0.075219,0.388090,-0.237690,-0.713239,...,0.294768,0.258762,-0.556602,-0.141297,-0.048803,0.073185,0.590476,0.000000,0.00000,0.073907
9996,-0.278203,0.118832,0.046464,0.021633,-0.015761,-0.449065,0.216599,0.531371,-0.190271,-0.481586,...,0.427946,0.280493,0.062318,-0.251321,0.094970,-0.245426,0.681159,0.000000,0.00000,0.082835
9997,-0.306959,0.368587,-0.233342,0.009264,-0.054013,-0.465737,-0.070932,0.556825,-0.231495,-0.611901,...,0.274999,0.065277,0.024031,-0.126560,0.074425,0.040631,0.605166,0.026667,0.02500,0.176306
9998,-0.342680,0.266963,-0.096156,-0.062876,-0.001338,-0.486167,-0.019108,0.476763,-0.234395,-0.396763,...,0.393987,0.133758,0.049314,-0.101994,-0.034232,-0.070724,0.881579,0.000000,0.00000,0.130269


In [263]:
main['rates'] = rates
main

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,kmpos,?,!,lens,rates
0,-0.591807,0.412227,-0.542035,0.235885,0.104132,-0.266516,-0.151623,0.747705,-0.491476,-0.921788,...,0.253669,-0.337376,-0.003289,-0.099390,0.081262,0.403433,0.013333,0.000000,0.045202,0.5
1,-0.561351,0.539296,-0.232209,0.017927,0.295141,-0.529878,-0.322349,0.504917,-0.184213,-0.902645,...,-0.012878,-0.207628,-0.119354,-0.248098,-0.024733,0.069106,0.013333,0.000000,0.043008,0.5
2,-0.664570,0.464939,-0.340457,-0.091051,0.005002,-0.515267,-0.082001,0.493307,-0.265265,-0.685733,...,0.016959,-0.493918,-0.072227,-0.121789,0.068586,0.328333,0.000000,0.000000,0.057782,0.5
3,-0.152027,0.341520,-0.116394,0.070313,-0.078539,-0.322824,-0.012664,0.391128,-0.158454,-0.564159,...,0.196713,-0.032280,-0.036533,0.172501,-0.068884,0.747748,0.000000,0.010638,0.176931,0.5
4,-0.238262,0.324238,-0.217468,0.280634,-0.047191,-0.554459,-0.017087,0.435945,-0.343209,-0.658837,...,0.164682,-0.201933,-0.162726,-0.003239,-0.002631,0.841808,0.000000,0.000000,0.092232,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,-0.325537,0.395521,-0.091214,0.134563,-0.017822,-0.500446,-0.172351,0.707692,-0.150523,-0.528643,...,0.199509,-0.235859,-0.154411,-0.138025,-0.127265,0.730337,0.000000,0.003546,0.064292,0.5
39996,-0.414435,0.448172,-0.341371,0.036420,-0.018985,-0.482495,-0.136398,0.583454,-0.224770,-0.705551,...,0.107461,-0.075953,-0.074680,-0.047151,-0.090945,0.234783,0.000000,0.003546,0.073800,0.5
39997,-0.420794,0.516332,-0.284983,0.123140,-0.000968,-0.410312,0.183259,0.430604,-0.188347,-0.672189,...,0.133464,-0.024950,-0.188739,-0.064351,-0.027500,0.547406,0.000000,0.007092,0.080383,0.5
39998,-0.433937,0.879612,-0.465086,0.016599,0.007571,-0.308500,0.029983,0.500054,-0.164624,-0.930576,...,0.153089,-0.259771,-0.215287,0.010675,0.003528,0.372807,0.000000,0.010638,0.043739,0.5


In [264]:
mainT['rates'] = rates_t
mainT

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,kmpos,?,!,lens,rates
0,-0.422088,0.623924,-0.134895,-0.150155,-0.003067,-0.356807,0.017889,0.374306,-0.075294,-0.723016,...,0.230552,0.095048,0.075119,0.031687,-0.148939,0.156682,0.000000,0.00625,0.100303,0.5
1,-0.448920,0.411618,-0.356737,0.110654,0.058743,-0.595456,-0.110939,0.521182,-0.152328,-0.720275,...,0.237417,-0.427480,-0.190685,-0.121355,0.152633,0.814815,0.000000,0.00625,0.053334,0.5
2,-0.392351,0.520844,-0.190956,0.059784,-0.038225,-0.412573,0.072141,0.380138,-0.211477,-0.651279,...,0.236064,0.024850,0.020207,0.043733,-0.152428,0.156682,0.026667,0.00000,0.386849,0.5
3,-0.289658,0.418946,-0.278016,0.057252,0.037030,-0.454300,-0.038163,0.492555,-0.204482,-0.675188,...,0.141809,-0.125700,-0.091181,0.050232,-0.069573,0.405204,0.013333,0.00000,0.366509,0.5
4,-0.526020,0.570565,-0.370353,-0.012514,0.027801,-0.276075,0.084151,0.519893,-0.301597,-0.556995,...,0.037079,0.011471,-0.052075,-0.127436,-0.122433,0.375000,0.000000,0.03750,0.094636,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.513173,0.669504,-0.193019,0.080172,-0.174542,-0.482258,-0.075219,0.388090,-0.237690,-0.713239,...,0.258762,-0.556602,-0.141297,-0.048803,0.073185,0.590476,0.000000,0.00000,0.073907,0.5
9996,-0.278203,0.118832,0.046464,0.021633,-0.015761,-0.449065,0.216599,0.531371,-0.190271,-0.481586,...,0.280493,0.062318,-0.251321,0.094970,-0.245426,0.681159,0.000000,0.00000,0.082835,0.5
9997,-0.306959,0.368587,-0.233342,0.009264,-0.054013,-0.465737,-0.070932,0.556825,-0.231495,-0.611901,...,0.065277,0.024031,-0.126560,0.074425,0.040631,0.605166,0.026667,0.02500,0.176306,0.5
9998,-0.342680,0.266963,-0.096156,-0.062876,-0.001338,-0.486167,-0.019108,0.476763,-0.234395,-0.396763,...,0.133758,0.049314,-0.101994,-0.034232,-0.070724,0.881579,0.000000,0.00000,0.130269,0.5


Попробуем добавить степени

Заново разделяем трейн

In [265]:
scaler = MinMaxScaler()

mainmax = scaler.fit_transform(mainmax)
mainTmax = scaler.fit_transform(mainTmax)
mainmax=pd.DataFrame(mainmax)
mainTmax=pd.DataFrame(mainTmax)


mainmin = scaler.fit_transform(mainmin)
mainTmin = scaler.fit_transform(mainTmin)
mainmin=pd.DataFrame(mainmin)
mainTmin=pd.DataFrame(mainTmin)

In [266]:
mainTmax

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.687923,1.000000,0.802708,0.736206,0.828275,0.621821,0.496719,0.623746,0.555020,0.782128,...,0.614141,0.758291,0.298196,0.844389,0.726310,0.903098,0.746652,0.773539,0.821841,0.674595
1,0.511644,0.732071,0.654193,0.674286,0.382675,0.470814,0.444284,0.499322,0.579414,0.782128,...,0.695864,0.424752,0.466688,0.547868,0.862792,0.623233,0.456576,0.826628,0.268905,0.737643
2,0.875942,0.732071,0.882544,0.973634,0.576902,0.838108,1.000000,0.623746,0.687993,0.792714,...,0.805919,0.903138,0.588668,0.888126,0.862792,0.903098,0.982070,1.000000,0.931915,0.737643
3,1.000000,1.000000,0.878840,0.886428,1.000000,0.838108,0.541674,0.767126,0.609205,0.792714,...,0.871118,0.758291,0.718926,0.882542,0.843977,1.000000,0.982070,1.000000,1.000000,0.674595
4,0.635322,1.000000,0.660479,0.736206,0.538085,0.966802,0.689659,1.000000,0.687993,0.849405,...,0.871118,0.607170,0.256679,0.864957,1.000000,0.450914,0.839833,0.976611,0.401314,0.671124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.351710,0.732071,0.715885,0.674286,0.309865,0.792108,0.518544,0.377130,0.555020,0.711821,...,0.805919,0.903138,1.000000,0.705123,0.528452,0.850117,0.748861,0.773539,0.483507,0.583530
9996,0.618232,0.154721,0.569515,0.755668,0.469937,0.683762,0.637247,0.806102,0.580387,0.811340,...,0.544749,0.758291,0.702556,0.774673,0.824018,0.716425,0.982070,0.761859,0.931915,0.836976
9997,0.635322,1.000000,0.815627,1.000000,1.000000,1.000000,0.627967,0.833830,0.578529,0.632179,...,0.787524,0.728532,0.462521,0.882542,0.589683,0.903098,1.000000,0.734217,0.890409,0.817387
9998,0.497065,1.000000,0.571204,0.811498,0.469937,0.554912,1.000000,0.833830,0.578529,0.811340,...,0.714695,0.758291,0.420654,0.623739,0.589683,0.459106,0.723318,0.761859,0.821841,0.566591


In [267]:
mainT=mainT.join(mainTmin, how='left', lsuffix='_left', rsuffix='_right')

main=main.join(mainmin, how='left', lsuffix='_left', rsuffix='_right')


In [268]:
mainT=mainT.join(mainTmax, how='left', lsuffix='_left', rsuffix='_right')
main=main.join(mainmax, how='left', lsuffix='_left', rsuffix='_right')

In [269]:
mainT

,0_left,1_left,2_left,3_left,4_left,5_left,6_left,7_left,8_left,9_left,...,90,91,92,93,94,95,96,97,98,99
0,-0.422088,0.623924,-0.134895,-0.150155,-0.003067,-0.356807,0.017889,0.374306,-0.075294,-0.723016,...,0.614141,0.758291,0.298196,0.844389,0.726310,0.903098,0.746652,0.773539,0.821841,0.674595
1,-0.448920,0.411618,-0.356737,0.110654,0.058743,-0.595456,-0.110939,0.521182,-0.152328,-0.720275,...,0.695864,0.424752,0.466688,0.547868,0.862792,0.623233,0.456576,0.826628,0.268905,0.737643
2,-0.392351,0.520844,-0.190956,0.059784,-0.038225,-0.412573,0.072141,0.380138,-0.211477,-0.651279,...,0.805919,0.903138,0.588668,0.888126,0.862792,0.903098,0.982070,1.000000,0.931915,0.737643
3,-0.289658,0.418946,-0.278016,0.057252,0.037030,-0.454300,-0.038163,0.492555,-0.204482,-0.675188,...,0.871118,0.758291,0.718926,0.882542,0.843977,1.000000,0.982070,1.000000,1.000000,0.674595
4,-0.526020,0.570565,-0.370353,-0.012514,0.027801,-0.276075,0.084151,0.519893,-0.301597,-0.556995,...,0.871118,0.607170,0.256679,0.864957,1.000000,0.450914,0.839833,0.976611,0.401314,0.671124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.513173,0.669504,-0.193019,0.080172,-0.174542,-0.482258,-0.075219,0.388090,-0.237690,-0.713239,...,0.805919,0.903138,1.000000,0.705123,0.528452,0.850117,0.748861,0.773539,0.483507,0.583530
9996,-0.278203,0.118832,0.046464,0.021633,-0.015761,-0.449065,0.216599,0.531371,-0.190271,-0.481586,...,0.544749,0.758291,0.702556,0.774673,0.824018,0.716425,0.982070,0.761859,0.931915,0.836976
9997,-0.306959,0.368587,-0.233342,0.009264,-0.054013,-0.465737,-0.070932,0.556825,-0.231495,-0.611901,...,0.787524,0.728532,0.462521,0.882542,0.589683,0.903098,1.000000,0.734217,0.890409,0.817387
9998,-0.342680,0.266963,-0.096156,-0.062876,-0.001338,-0.486167,-0.019108,0.476763,-0.234395,-0.396763,...,0.714695,0.758291,0.420654,0.623739,0.589683,0.459106,0.723318,0.761859,0.821841,0.566591


In [270]:
main

,0_left,1_left,2_left,3_left,4_left,5_left,6_left,7_left,8_left,9_left,...,90,91,92,93,94,95,96,97,98,99
0,-0.591807,0.412227,-0.542035,0.235885,0.104132,-0.266516,-0.151623,0.747705,-0.491476,-0.921788,...,0.806796,0.676658,0.776087,0.866363,0.735299,0.815005,0.979675,0.762720,0.881198,0.677107
1,-0.561351,0.539296,-0.232209,0.017927,0.295141,-0.529878,-0.322349,0.504917,-0.184213,-0.902645,...,0.849037,0.676658,0.554620,0.581398,0.892269,0.491856,0.654196,0.469294,0.684466,0.714381
2,-0.664570,0.464939,-0.340457,-0.091051,0.005002,-0.515267,-0.082001,0.493307,-0.265265,-0.685733,...,0.778079,0.918696,0.771336,0.790731,1.000000,0.671467,0.686355,0.975031,0.881198,0.896497
3,-0.152027,0.341520,-0.116394,0.070313,-0.078539,-0.322824,-0.012664,0.391128,-0.158454,-0.564159,...,0.702304,0.918696,0.683783,0.811220,0.676742,0.917733,0.972772,1.000000,0.926921,0.858418
4,-0.238262,0.324238,-0.217468,0.280634,-0.047191,-0.554459,-0.017087,0.435945,-0.343209,-0.658837,...,0.784930,0.592214,0.517988,0.856735,0.628194,0.767130,0.923723,0.671813,0.658012,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,-0.325537,0.395521,-0.091214,0.134563,-0.017822,-0.500446,-0.172351,0.707692,-0.150523,-0.528643,...,0.849037,0.664298,0.668810,0.771727,0.735299,0.759253,0.979675,0.777357,0.818622,0.677107
39996,-0.414435,0.448172,-0.341371,0.036420,-0.018985,-0.482495,-0.136398,0.583454,-0.224770,-0.705551,...,0.849037,0.918696,0.519424,0.898422,0.684676,0.552226,0.818435,0.921763,0.881198,0.751986
39997,-0.420794,0.516332,-0.284983,0.123140,-0.000968,-0.410312,0.183259,0.430604,-0.188347,-0.672189,...,0.849037,0.674878,1.000000,0.817753,0.688839,0.872754,0.979675,0.921763,0.881198,0.677968
39998,-0.433937,0.879612,-0.465086,0.016599,0.007571,-0.308500,0.029983,0.500054,-0.164624,-0.930576,...,0.813486,1.000000,0.611609,0.646909,0.624922,0.716803,0.608987,0.758247,0.745864,0.420350


SVM

In [280]:
X_train, X_test, y_train, y_test = train_test_split(main, df.label.values,
                                                    test_size=0.2, shuffle=True)

In [281]:
from sklearn import svm

In [282]:
clf = svm.LinearSVC().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [283]:
predicts = clf.predict(X_train)
print('Train\n', classification_report(y_train, predicts))

predicts = clf.predict(X_test)
print('Test\n', classification_report(y_test, predicts))

Train
               precision    recall  f1-score   support

           0       0.87      0.86      0.87     16076
           1       0.86      0.87      0.87     15924

    accuracy                           0.87     32000
   macro avg       0.87      0.87      0.87     32000
weighted avg       0.87      0.87      0.87     32000

Test
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      3991
           1       0.86      0.86      0.86      4009

    accuracy                           0.86      8000
   macro avg       0.86      0.86      0.86      8000
weighted avg       0.86      0.86      0.86      8000



In [284]:
f1_score(y_test,predicts)

0.862013189000871

In [285]:
clf = svm.LinearSVC().fit(main,labels)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [286]:
predicts = clf.predict(mainT)

In [287]:
len(np.where(predicts ==1)[0])

5431

In [288]:
pd.DataFrame({'Predicted': predicts}).to_csv('w2vec15.csv', index_label='Id')

In [289]:
len(predicts)

10000

Регрессия

In [290]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)

In [291]:
from sklearn.metrics import classification_report

predicts = clf.predict(X_train)
print('Train\n', classification_report(y_train, predicts))

predicts = clf.predict(X_test)
print('Test\n', classification_report(y_test, predicts))

Train
               precision    recall  f1-score   support

           0       0.87      0.86      0.87     16076
           1       0.86      0.87      0.87     15924

    accuracy                           0.87     32000
   macro avg       0.87      0.87      0.87     32000
weighted avg       0.87      0.87      0.87     32000

Test
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      3991
           1       0.86      0.87      0.86      4009

    accuracy                           0.86      8000
   macro avg       0.86      0.86      0.86      8000
weighted avg       0.86      0.86      0.86      8000



In [292]:
f1_score(y_test,predicts)

0.8640099626400997

In [468]:
#f1res = np.zeros(100)

In [469]:
#f1res[1] = f1_score(y_test,predicts)

In [470]:
#f1res

То же для ответа

In [295]:
clf = LogisticRegression(solver='lbfgs', max_iter=1000).fit(main, labels)
predicts = clf.predict(mainT)

pd.DataFrame({'Predicted': predicts}).to_csv('w2vec15lr.csv', index_label='Id')

In [294]:
len(np.where(predicts ==1)[0])

5395